In [0]:
''' 
Este script permite descargar los datos de la base de datos de firebase. Estos datos, ya que vienen en formato JSON, 
también se transforman a formato csv. 
'''
#Permite acceder a los archivos de google drive 

from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#instalación de librerías necesarias
!pip install firebase_admin

     |████████████████████████████████| 92kB 3.1MB/s 
     |████████████████████████████████| 337kB 8.8MB/s 
     |████████████████████████████████| 81kB 11.7MB/s 
ERROR: google-cloud-storage 1.26.0 has requirement google-auth<2.0dev,>=1.11.0, but you'll have google-auth 1.7.2 which is incompatible.
ERROR: google-cloud-storage 1.26.0 has requirement google-cloud-core<2.0dev,>=1.2.0, but you'll have google-cloud-core 1.0.3 which is incompatible.
ERROR: google-cloud-bigquery 1.21.0 has requirement google-resumable-media!=0.4.0,<0.5.0dev,>=0.3.1, but you'll have google-resumable-media 0.5.0 which is incompatible.
  Found existing installation: google-resumable-media 0.4.1
    Uninstalling google-resumable-media-0.4.1:
      Successfully uninstalled google-resumable-media-0.4.1
  Found existing installation: google-cloud-storage 1.16.2
    Uninstalling google-cloud-storage-1.16.2:
      Successfully uninstalled google-cloud-storage-1.16.2


In [0]:
import csv
import pandas as pd
from datetime import datetime
import json, csv
import numpy as np
import firebase_admin
import json
from firebase_admin import credentials
from firebase_admin import db	

In [0]:
#buscamos en google drive la carpeta en la que queremos guardar los datos
cd '/content/drive/My Drive/Proyecto 2 SENSORES/Ficheros de datos/'

/content/drive/My Drive/Proyecto 2 SENSORES/Ficheros de datos


In [0]:
#Esta función nos permite descargar los datos de firebase y generar un JSON (nos lo pasó senen)

def downloadFirebaseDatabase(options = "all", t_start = 0, t_end = 0,filename = 'dataRetrived.json'):
  """
  Obtenemos datos de la base de datos y los guardamos con un objeto json
  Según eel parámetro de options que le pasemos realizará una tarea o otra
  options = "all" --> Nos devuelve toda la base de datos predeterminado
  options = "timestamp" --> Nos devuelve los datos que estén el timestamp seleccionado
                            en esta opción habrá introducir como parámetros el tiempo que
                            queramos. En caso, de que en ese intervalo no haya nigún dato
                            no se creeará ningún objeto json.
  """

  if not len(firebase_admin._apps):
    cred = credentials.Certificate("/content/drive/My Drive/Proyecto 2 SENSORES/Scripts/Private keys/PrivateKeyFirebase.json")

    firebase_admin.initialize_app(cred, {'databaseURL': 'https://proyecto-sensores-48694.firebaseio.com/'})

  data = db.reference()

  if options =='timestamp':
    while 1:
        try:
            TIMESTAMP_FROM = int(t_start)
            break
        except ValueError:
            print("Enter only digit.")
            continue

    while 1:
        try:
            TIMESTAMP_TO = int(t_end)
            break
        except ValueError:
            print("Enter only digit.")
            continue

    result = data.order_by_child('timestamp').start_at(TIMESTAMP_FROM).end_at(TIMESTAMP_TO)
  else:
    result = data
    d = result.get();

  path = '/content/drive/My Drive/Proyecto 2 SENSORES/Ficheros de datos/'
  print(path+filename)
  with open(path+filename,'w') as o:
      json.dump(d, o)

downloadFirebaseDatabase(filename='datosFirebase.json')

/content/drive/My Drive/Proyecto 2 SENSORES/Ficheros de datos/datosFirebase.json


In [0]:
#Esta función nos permite coger el fichero JSON descargado en el script anterior y convertirlo a formato CSV

def firebaseToCsv(filename):
  ##Filename sin extensión
  ## firebasedata -> bien
  ## firebasedata.json -> mal
  with open(filename+'.json') as f:
      datos = json.load(f)
  listaGrande = []
  for i in datos: #datos solo tiene una clave-valor :  Data:{} y ese diccionario grande tiene pares 'cod_medida1': {"C":693,"H":36..}
      for medida in datos[i]: #recorremos el diccionario grande con todas las medidas
          lista = []
          for pos,key in enumerate(datos[i][medida]): #recorremos las claves de los 6 valores tomados en cada medida : {"C": ,"H": , ...}
              if pos == 5: #clave timestamp
                valor = datos[i][medida][key] #es el tiempo que ha pasado desde que han creado UNIX, hay que transformarlo
                valor = valor/1000
                valor += 3600                 #le sumamos una hora en segundos para pasarlo a hora espanyola 
                buen_valor = datetime.fromtimestamp(valor)
                buen_valor = str(buen_valor).split('.')[0]
                datos[i][medida][key] = buen_valor
              lista.append(datos[i][medida][key]) #'lista'solo contiene los 6 valores de cada medida
          listaGrande.append(lista)


  with open(filename+'.csv', mode='w') as file:
      file_writer = csv.writer(file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
      file_writer.writerow(['CO2', 'Humedad', 'Ruido', 'NOx', 'Temperatura', 'Timestamp']) #añadimos el header al csv
      for i in listaGrande:
          file_writer.writerow(i)
  return

firebaseToCsv('datosFirebase')